In [1]:
import pandas as pd
import json
import fasttext
import emoji
import re
import collections
from urllib.parse import urlparse

In [193]:
df_posts = pd.read_csv('/Users/katerynaburovova/PycharmProjects/dehumanization/only_text.csv', header=0)

In [194]:
df_posts

,id,date,message,type,channel_name,lang
0,12347.0,2022-12-15 16:32:15+00:00,А трансляция тем временем идет. Подписывайтесь...,photo,mardanaka,ru
1,12346.0,2022-12-15 15:00:03+00:00,"В России ещё есть те, кто способен возрождать ...",photo,mardanaka,ru
2,12345.0,2022-12-15 14:21:22+00:00,«Катаргейт» набирает обороты - Обвиняемая в ко...,text,mardanaka,ru
3,12344.0,2022-12-15 13:08:35+00:00,🔥В гостях у @Metametrica побывали товарищи из ...,photo,mardanaka,ru
4,12343.0,2022-12-15 12:31:23+00:00,Глава Минобороны Грузии назвал грузинских наем...,text,mardanaka,ru
...,...,...,...,...,...,...
6836539,8.0,2018-10-18 14:18:13+00:00,Мария Топилина – муж – Максим Топилин – минист...,text,bloodysx,ru
6836540,7.0,2018-10-18 14:17:44+00:00,"Людмила Красноштанова – муж – член «ЕдРа», деп...",text,bloodysx,ru
6836541,6.0,2018-10-18 14:16:00+00:00,"И, на всякий случай, если вы все проспали, дуб...",text,bloodysx,ru
6836542,5.0,2018-10-18 14:00:26+00:00,"Вот вам доказательство, что это реальная я.",video,bloodysx,ru


In [195]:
df_sample = df_posts.sample(frac=0.2, random_state=42).copy()
df_sample.reset_index(inplace=True)

In [196]:
df_small_sample = df_posts.sample(frac=0.1, random_state=42).copy()
df_small_sample.reset_index(inplace=True)

In [20]:
import tokenize_uk

In [23]:
len(df_sample)

1367309

In [333]:
%%time

df_sample['tokenized_sents'] = df_sample['message'].apply(lambda x: tokenize_uk.tokenize_sents(x))

CPU times: user 36 s, sys: 23.6 s, total: 59.6 s
Wall time: 1min 45s


In [28]:
df_sample['tokenized_sents'][2000]

['#АлтайАлтаюРознь2 Не утихают федеральные телеграм-скандалы с упоминанием губернатора Алтайского края Карлина.',
 'В соседней Республике Алтай ситуация более стабильна.',
 'Глава региона вплотную занялся проблемами обманутых дольщиков и накануне провел встречу с пострадавшими от мошенников гражданами.',
 'После их обращения генерал Бердников изыскал средства в республиканском бюджете (46 млн) и на встрече с жителями заверил, что дом по улице Чаптынова, 34 в Горно-Алтайске на 60 квартир будет сдан уже в следующем году.',
 'Кстати, директор строительной фирмы «ГАСК» Борщев уже отбывает наказание в виде 6 лет лишения свободы.   http://www.interfax-russia.ru/Siberia/news.asp?id=911826&sec=1671']

## We set aside 1000 neutral sentences (not expected to contain neither dehumanization, not even mentions of Ukrainians)

In [31]:
df_sample['random_sents'] = df_sample.tokenized_sents.str[0]

In [36]:
neutral_sent_list = df_sample['random_sents'].sample(1000, random_state=42).tolist()

In [37]:
neutral_sent_list

['Осиротевших из-за пожара в Австралии кенгурят выкармливают в приюте.',
 '❗️Высший совет ЛДПР выдвинул в лидеры партии Слуцкого (решение примет завтрашний партийный съезд)',
 '\u200b«Ох, как пригодилась бы сейчас кое-кому Третья Мировая - прикрыла бы позор.',
 'Две тысячи пятьсот тридцать восемь лет  Именно столько Максим Товкайло будет выплачивать долг «Роснефти», если требуемую сумму компенсации разделить между ответчиками пропорционально.',
 'Мы ведь всё равно не будем читать эти надписи мелким шрифтом, правда же? 😑ѣѣ',
 'В Иране впервые представлен аналог ударного БЛА MQ-9 Reaper «Армейский лук 22»',
 '«Мемориал» штампует школьников, жалеющих фашистов, при этом ворует из архива и выкладывает в открытый доступ базу с персональными данными сотрудников НКВД. Либеральная общественность воет и лезет из шкуры.',
 'Семен Шитиков про анимацию в России',
 'Еще одно свидетельство применения на Украине комплекса космической интернет-связи Starlink в военных целях: американский наемник благод

In [341]:
df_neutral = pd.DataFrame({'text': neutral_sent_list[:]})
df_neutral.to_csv('df_neutral.csv')

In [64]:
df_sample['random_sents'][6310]

'Четверых украинцев конвоировали в «Лефортово».'

In [65]:
df_sample['tokenized_sents'][6310]

['Четверых украинцев конвоировали в «Лефортово».',
 'Их обвиняют в международном терроризме из-за подготовки теракта против гуманитарного конвоя в Мелитополе.',
 'В московском изоляторе содержатся председатель федерации кунг-фу Мелитополя Андрей Голубев* (на фото), жители города Александр Жуков*, Игорь Голубев* и Владимир Зуев*, которых задержали 6 апреля, а также уроженец Запорожья Юрий Петров — он пропал 1 апреля.',
 'Их обвиняют по статье об акте международного терроризма. 22 апреля ФСБ сообщила о предотвращении теракта в Мелитополе: по версии силовиков, диверсанты заложили 8 кг пластида в передний бампер автомобиля, чтобы взорвать его в 5 метрах от российского гуманитарного конвоя.',
 'Через неделю мужчин внесли в реестр Росфинмониторинга в качестве террористов.',
 'Расследованием дела занимается Следственное управление ФСБ — мужчин арестовывали в Крыму, но теперь аресты фигурантам продлевает Лефортовский суд.',
 'Им грозит пожизненный срок.  *внесены в реестр террористов Росфинмон

## Selecting the sentences that contain mentions of Ukraine, ukrainians

In [38]:
seqs_to_track = ['укрa', 'хох', 'хохл']

In [335]:
def find_sentences_w_sec(list_sent, seqs_to_track):
    sent_list_result = []
    for sent in list_sent:
        for seq in seqs_to_track:
            if seq in sent.lower():
                sent_list_result.append(sent)
    return sent_list_result

In [87]:
%%time
find_sentences_w_sec(df_sample['tokenized_sents'][6310], seqs_to_track)

CPU times: user 48 µs, sys: 1e+03 ns, total: 49 µs
Wall time: 48.9 µs


['Четверых украинцев конвоировали в «Лефортово».']

In [89]:
%%time
df_sample['mention_ukr'] = df_sample['tokenized_sents'].apply(lambda x: find_sentences_w_sec(x, seqs_to_track))

CPU times: user 2.88 s, sys: 1.71 s, total: 4.59 s
Wall time: 4.98 s


In [117]:
data = df_sample['mention_ukr'].explode().unique()

In [125]:
df_mentions = pd.DataFrame({'text': data[:]})

In [126]:
df_mentions.to_csv('ukr_mentions.csv', index = False, header=True)

In [127]:
import torch
from transformers import AutoModelForSequenceClassification
from transformers import BertTokenizerFast

In [128]:
tokenizer = BertTokenizerFast.from_pretrained('blanchefort/rubert-base-cased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('blanchefort/rubert-base-cased-sentiment', return_dict=True)


def predict(text):
    inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt')
    outputs = model(**inputs)
    predicted = torch.nn.functional.softmax(outputs.logits, dim=1)
    predicted = torch.argmax(predicted, dim=1).numpy()
    return predicted

Downloading:   0%|          | 0.00/499 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/943 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/712M [00:00<?, ?B/s]

### Trying with more radical wording

In [165]:
str_to_track = ['укры', 'хох', 'хохл', 'рейх', 'фаш']

In [166]:
%%time
df_sample['mention_ukr_radical'] = df_sample['tokenized_sents'].apply(lambda x: find_sentences_w_sec(x, str_to_track))

CPU times: user 4.51 s, sys: 17.5 s, total: 22 s
Wall time: 53.4 s


In [168]:
data_radical = df_sample['mention_ukr_radical'].explode().unique()
df_radical = pd.DataFrame({'text': data_radical[:]})

In [177]:
%%time
df_radical = df_radical[1:]

CPU times: user 104 µs, sys: 141 µs, total: 245 µs
Wall time: 227 µs


In [180]:
%%time
df_radical['tone'] = df_radical['text'].apply(lambda x: predict(x))

CPU times: user 12min 30s, sys: 3min 33s, total: 16min 4s
Wall time: 10min 20s


<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [185]:
df_radical = df_radical[df_radical['tone']==2]

In [186]:
df_radical.to_csv('df_radical.csv', index = False, header=True)

## Selecting the posts that contain mentions of Ukraine, ukrainians

In [131]:
df_sample

,level_0,index,id,date,message,type,channel_name,lang,tokenized_sents,random_sents,mention_ukr
0,0,4682808,77377.0,2022-09-12 10:15:03+00:00,🤝Дмитрий Песков: В победе врио губернаторов на...,text,HouseOfCardsRussia,ru,[🤝Дмитрий Песков: В победе врио губернаторов н...,🤝Дмитрий Песков: В победе врио губернаторов на...,[]
1,1,19359,178850.0,2022-09-22 09:09:34+00:00,МИД: Россия к малейшему признаку улучшения отн...,text,rian_ru,ru,[МИД: Россия к малейшему признаку улучшения от...,МИД: Россия к малейшему признаку улучшения отн...,[]
2,2,3479193,41197.0,2022-04-07 14:10:02+00:00,"🗳 Санкции сказались на работе фирмы Faberlic, ...",photo,stormdaily,ru,"[🗳 Санкции сказались на работе фирмы Faberlic,...","🗳 Санкции сказались на работе фирмы Faberlic, ...",[]
3,3,318948,21532.0,2022-06-19 06:04:07+00:00,Новая Барби уже не та. Содомиты и демоны.,photo,pravda_shuravi,ru,"[Новая Барби уже не та., Содомиты и демоны.]",Новая Барби уже не та.,[]
4,4,1882232,19450.0,2019-06-06 10:21:11+00:00,💥 Эксклюзив 360° Пообщались с Мутко на полях...,photo,tv360,ru,[💥 Эксклюзив 360° Пообщались с Мутко на поля...,💥 Эксклюзив 360° Пообщались с Мутко на полях...,[]
...,...,...,...,...,...,...,...,...,...,...,...
1367304,1367304,1210683,43526.0,2021-06-22 12:56:35+00:00,Российского легкоатлета Сергея Шубенкова оправ...,photo,meduzalive,ru,[Российского легкоатлета Сергея Шубенкова опра...,Российского легкоатлета Сергея Шубенкова оправ...,[]
1367305,1367305,4452914,56010.0,2022-09-25 16:50:18+00:00,"Сообщается, что в результате взрыва автомобиля...",video,ru2ch_news,ru,"[Сообщается, что в результате взрыва автомобил...","Сообщается, что в результате взрыва автомобиля...","[По информации властей, это был украинский див..."
1367306,1367306,5243658,5973.0,2019-09-14 15:42:48+00:00,Петербургские москвичи и московские петербуржц...,video,bogemasranaya,ru,[Петербургские москвичи и московские петербурж...,Петербургские москвичи и московские петербуржц...,[]
1367307,1367307,977011,32166.0,2022-09-25 12:07:59+00:00,«В своем выступлении в ООН я задал простой воп...,photo,ErnestV_2020,ru,[«В своем выступлении в ООН я задал простой во...,«В своем выступлении в ООН я задал простой воп...,[]


In [135]:
strings = ["укр", "Укр" "хох", "Хох"]
df_filtered = df_sample[df_sample["message"].str.contains('|'.join(strings))]

In [137]:
len(df_sample)

1367309

In [142]:
df_tone_sample = df_filtered.sample(10000, random_state=42).copy()

In [143]:
%%time
df_tone_sample['tone'] = df_tone_sample['message'].apply(lambda x: predict(x))

CPU times: user 27min 45s, sys: 7min, total: 34min 45s
Wall time: 15min 21s


In [146]:
df_tone_sample[df_tone_sample['tone']==2]

,level_0,index,id,date,message,type,channel_name,lang,tokenized_sents,random_sents,mention_ukr,tone
1269901,1269901,817024,166442.0,2022-11-15 22:48:27+00:00,Что известно о взрывах на востоке Польши ◾️Ве...,text,tass_agency,ru,[Что известно о взрывах на востоке Польши ◾️В...,Что известно о взрывах на востоке Польши ◾️Ве...,"[В министерстве заявили, что ударов российским...",[2]
640138,640138,1179620,1127.0,2021-04-15 15:08:05+00:00,"Российский военкор рассказал, когда ждать пров...",photo,swodki,ru,"[Российский военкор рассказал, когда ждать про...","Российский военкор рассказал, когда ждать пров...","[Российский военкор рассказал, когда ждать про...",[2]
225021,225021,5409015,45661.0,2022-04-28 13:36:06+00:00,"♨️ Украинские Т-каналы пишут, что в Киеве и Од...",video,boris_rozhin,ru,"[♨️ Украинские Т-каналы пишут, что в Киеве и О...","♨️ Украинские Т-каналы пишут, что в Киеве и Од...","[В украинских чатах пишут, что виной всему все...",[2]
620894,620894,1050852,197532.0,2022-12-01 21:05:20+00:00,Фронтовая сводка в направлении Сватово - Креме...,video,swodki,ru,[Фронтовая сводка в направлении Сватово - Крем...,Фронтовая сводка в направлении Сватово - Креме...,[Фронтовая сводка в направлении Сватово - Крем...,[2]
450236,450236,1290156,47765.0,2022-11-27 16:00:01+00:00,Украинская энергосистема пожирает сама себя: ч...,photo,readovkanews,ru,[Украинская энергосистема пожирает сама себя: ...,Украинская энергосистема пожирает сама себя: ч...,[Украинская энергосистема пожирает сама себя: ...,[2]
...,...,...,...,...,...,...,...,...,...,...,...,...
303016,303016,2794715,9437.0,2022-02-27 13:46:26+00:00,⚡️⚡️⚡️СРОЧНО МИНОБОРОНЫ РОССИИ Украинские во...,text,skabeeva,ru,[⚡️⚡️⚡️СРОЧНО МИНОБОРОНЫ РОССИИ Украинские в...,⚡️⚡️⚡️СРОЧНО МИНОБОРОНЫ РОССИИ Украинские во...,[⚡️⚡️⚡️СРОЧНО МИНОБОРОНЫ РОССИИ Украинские в...,[2]
75558,75558,3846260,157781.0,2022-03-27 15:49:05+00:00,⚡️Глава СК РФ Александр Бастрыкин поручил расс...,text,glavmedia,ru,[⚡️Глава СК РФ Александр Бастрыкин поручил рас...,⚡️Глава СК РФ Александр Бастрыкин поручил расс...,[⚡️Глава СК РФ Александр Бастрыкин поручил рас...,[2]
597630,597630,1096190,130503.0,2022-07-09 07:10:02+00:00,Основные итоги СВО на Украине за 8 июля 2022 г...,video,swodki,ru,[Основные итоги СВО на Украине за 8 июля 2022 ...,Основные итоги СВО на Украине за 8 июля 2022 г...,[Еще одно нападение украинских формирований на...,[2]
733911,733911,2235534,24801.0,2022-09-01 18:00:00+00:00,⏱ Главные события на 21:00 ▪️Делегация МАГАТЭ...,text,OstashkoNews,ru,[⏱ Главные события на 21:00 ▪️Делегация МАГАТ...,⏱ Главные события на 21:00 ▪️Делегация МАГАТЭ...,[⏱ Главные события на 21:00 ▪️Делегация МАГАТ...,[2]


In [162]:
data2 = df_tone_sample['tokenized_sents'][df_tone_sample['tone']==2].explode().unique()
df_ukr_neg = pd.DataFrame({'text': data2[:]})
df_ukr_neg.to_csv('ukr_neg.csv', index = False, header=True)

In [149]:
strings2 = ["укры", "Укры" "хох", "Хох"]
df_filtered2 = df_sample[df_sample["message"].str.contains('|'.join(strings))]

In [150]:
len(df_filtered2)

94927

In [151]:
df_tone_sample2 = df_filtered2.sample(5000, random_state=42).copy()

In [152]:
df_tone_sample2['tone'] = df_tone_sample2['message'].apply(lambda x: predict(x))

In [160]:
data3 = df_tone_sample2['tokenized_sents'][df_tone_sample2['tone']==2].explode().unique()
df_ukr_neg2 = pd.DataFrame({'text': data3[:]})
df_ukr_neg2.to_csv('ukr_neg2.csv', index = False, header=True)

### For w2v training

In [197]:
df_small_sample

,index,id,date,message,type,channel_name,lang
0,4682808,77377.0,2022-09-12 10:15:03+00:00,🤝Дмитрий Песков: В победе врио губернаторов на...,text,HouseOfCardsRussia,ru
1,19359,178850.0,2022-09-22 09:09:34+00:00,МИД: Россия к малейшему признаку улучшения отн...,text,rian_ru,ru
2,3479193,41197.0,2022-04-07 14:10:02+00:00,"🗳 Санкции сказались на работе фирмы Faberlic, ...",photo,stormdaily,ru
3,318948,21532.0,2022-06-19 06:04:07+00:00,Новая Барби уже не та. Содомиты и демоны.,photo,pravda_shuravi,ru
4,1882232,19450.0,2019-06-06 10:21:11+00:00,💥 Эксклюзив 360° Пообщались с Мутко на полях...,photo,tv360,ru
...,...,...,...,...,...,...,...
683649,869482,112120.0,2022-02-24 07:49:18+00:00,Франция запрещает своим самолетам полеты в воз...,text,tass_agency,ru
683650,3504955,9412.0,2019-07-04 05:41:19+00:00,Возможной причиной гибели 14 моряков на глубок...,text,stormdaily,ru
683651,4914297,3940.0,2019-11-18 13:12:15+00:00,Сегодня нам исполнился месяц. С чем мы и поздр...,text,polittemnik,ru
683652,616522,29331.0,2021-10-14 10:57:04+00:00,Стрелка Тимура Бекмансурова арестовали на два ...,photo,sotaproject,ru


In [198]:
len(df_small_sample)

683654

In [199]:
splitters = ['\n', '\t', '\\n','\xa0', '\u200b']
df_small_sample['message_no_breaks'] = df_small_sample['message'].str.replace('\n|\t|\\n', ' ', case=False)

/var/folders/2w/hygxgpp5353_dy_tk88gw2100000gn/T/ipykernel_32512/2069008346.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_small_sample['message_no_breaks'] = df_small_sample['message'].str.replace('\n|\t|\\n', ' ', case=False)


In [200]:
def get_emoji_count(text):
    return collections.Counter([match["emoji"] for word in text for match in emoji.emoji_list(word)])

In [201]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r' ', string)

In [202]:
%%time
df_small_sample['message_no_emoji'] = df_small_sample['message_no_breaks'].apply(lambda x: remove_emoji(x))

CPU times: user 4.92 s, sys: 7.77 s, total: 12.7 s
Wall time: 14.9 s


In [203]:
df_small_sample['message_no_urls'] = df_small_sample['message_no_emoji'].str.replace('http\S+|www.\S+', '', case=False)

/var/folders/2w/hygxgpp5353_dy_tk88gw2100000gn/T/ipykernel_32512/1803551423.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_small_sample['message_no_urls'] = df_small_sample['message_no_emoji'].str.replace('http\S+|www.\S+', '', case=False)


In [204]:
def cut_mentions(list_of_mentions, text):
    for word in list_of_mentions:
        wrd = '@' + word
        text = text.replace(wrd, '')
    return text.strip()

In [206]:
df_small_sample['mentions'] = df_small_sample['message_no_urls'].str.findall(r"@([a-zA-Z0-9_]{1,50})")

In [207]:
%%time
df_small_sample['message_no_mentions'] = df_small_sample.apply(lambda x: cut_mentions(x['mentions'], x['message_no_urls']), axis=1)

CPU times: user 5.7 s, sys: 47.5 s, total: 53.2 s
Wall time: 1min 11s


In [208]:
to_replace = ["«", "»", "“"]

df_small_sample['message_no_mentions'] = df_small_sample['message_no_mentions'].str.replace('|'.join(to_replace),'"')

/var/folders/2w/hygxgpp5353_dy_tk88gw2100000gn/T/ipykernel_32512/195567455.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df_small_sample['message_no_mentions'] = df_small_sample['message_no_mentions'].str.replace('|'.join(to_replace),'"')


In [209]:
def cut_hasgtags(set_of_hashtags, text):
    for word in list(set_of_hashtags):
        text = text.replace(word, '')
    return text.strip()

In [211]:
%%time
df_small_sample['hashtags'] = df_small_sample['message_no_mentions'].apply(lambda x: {word.strip(",").strip(".") for word in x.split() if word.startswith("#")})

CPU times: user 4.38 s, sys: 1.31 s, total: 5.69 s
Wall time: 6.34 s


In [212]:
df_small_sample['message_no_hashtags'] = df_small_sample.apply(lambda x: cut_hasgtags(x['hashtags'], x['message_no_mentions']), axis=1)


In [213]:
punctuation_minimal = '!(),-.:;?%'

cyrillic_letters = u"абвгдеёжзийклмнопрстуфхцчшщъыьэюяАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ"
latin_letters_numbers = "1234567890abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ "
allowed_symbols = cyrillic_letters+latin_letters_numbers + punctuation_minimal

In [214]:
def clean_text(string, allowed_symbols=allowed_symbols):
    getVals = list(filter(lambda x: x in allowed_symbols, string))
    result = "".join(getVals)
    result = re.sub(' +', ' ', result)

    return result

In [215]:
%%time
df_small_sample['message_clean'] = df_small_sample['message_no_hashtags'].apply(lambda x: clean_text(x))

CPU times: user 25.7 s, sys: 2.73 s, total: 28.4 s
Wall time: 29.1 s


In [216]:
only_letters = cyrillic_letters+latin_letters_numbers

In [217]:
%%time
df_small_sample['message_words_only_lower'] = df_small_sample['message_clean'].apply(lambda x: clean_text(x, allowed_symbols=only_letters).lower())

CPU times: user 24.8 s, sys: 520 ms, total: 25.3 s
Wall time: 25.4 s


In [235]:
with pd.option_context('display.max_colwidth', None):
    print(df_small_sample['message_words_only_lower'].sample(1))

438423    более 400 участников межвузовского чемпионата worldskills встретятся на финале в москве в соревнованиях примут участие более 400 конкурсантов и 500 экспертов из 80 вузов в том числе из других стран впервые на чемпионате будут соревноваться студенты и эксперты из зарубежных университетов в том числе из таких стран как белоруссия германия казахстан китай марокко хорватия эстония финал межвузовского чемпионата молодые профессионалы движения worldskills пройдет в москве с 26 по 29 ноября
Name: message_words_only_lower, dtype: object


In [238]:
df_small_sample[['message_words_only_lower']].to_csv('small_sample.csv', header=True)

In [239]:
import spacy

In [240]:
nlp = spacy.load('ru_core_news_md')

In [241]:
def tokenize_spacy(text):
    doc = nlp(text)
    result = []
    for token in doc:
        result.append([token.text, token.pos_, token.dep_, token.lemma_, token.morph])
    return result

In [242]:
%%time
df_small_sample['tokens'] = df_small_sample['message_words_only_lower'].apply(lambda x: tokenize_spacy(x))

CPU times: user 3h 14min 18s, sys: 4min 21s, total: 3h 18min 39s
Wall time: 1d 16h 53min 52s


In [243]:
df_small_sample['tokens']

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [244]:
df_small_sample.to_csv('df_small_sample.csv')

In [245]:
def tokenize_spacy_lemma(text):
    doc = nlp(text)
    result = []
    for token in doc:
        result.append(token.lemma_)
    return result

In [263]:
%%time
test_set = df_small_sample['tokens'].sample(1000).apply(lambda x: get_lemma(x))

CPU times: user 25 ms, sys: 215 ms, total: 240 ms
Wall time: 1.2 s


In [256]:
df_small_sample['tokens'][0][0][3]

'дмитрий'

In [261]:
def get_lemma(list_of_lists):
    result = []
    for list in list_of_lists:
        result.append(list[3])
    return result

In [265]:
%%time
df_small_sample['lemmas'] = df_small_sample['tokens'].apply(lambda x: get_lemma(x))

CPU times: user 3.71 s, sys: 11 s, total: 14.7 s
Wall time: 53.4 s


In [266]:
df_small_sample['lemmas']

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     [дмитрий, песков, в, победа, врио, губернатор, на, выбор, быть, заслуга, президент, рф, который, поддержать, кандидат]
1                                                                                                                                                                          

In [267]:
documents = df_small_sample['lemmas'].tolist()

In [270]:
import gensim

In [273]:
%%time
model = gensim.models.Word2Vec (documents, vector_size=150, window=10, min_count=2, workers=10)
model.train(documents,total_examples=len(documents),epochs=10)

CPU times: user 21min 19s, sys: 1min 55s, total: 23min 14s
Wall time: 6min 48s


(333227208, 381780410)

In [282]:
w1 = "украинец" #ы
model.wv.most_similar (positive=w1)

[('поляк', 0.7101138234138489),
 ('русских', 0.7060922980308533),
 ('русские', 0.683661699295044),
 ('европеец', 0.6558117866516113),
 ('немец', 0.6476935744285583),
 ('американец', 0.6276693344116211),
 ('бандеровец', 0.6234453320503235),
 ('британец', 0.6072903275489807),
 ('украина', 0.6048406362533569),
 ('киевлянин', 0.6004078388214111)]

In [294]:
w2 = "украинцы" #ы
model.wv.most_similar (positive=w2)

KeyError: "Key 'украинцы' not present in vocabulary"

In [295]:
w2 = "укронацисты" #ы
model.wv.most_similar (positive=w2)

[('всушники', 0.7371035814285278),
 ('всу', 0.7126280069351196),
 ('укры', 0.6975746154785156),
 ('укроп', 0.6604259014129639),
 ('украинские', 0.6520400643348694),
 ('укровояки', 0.6500487923622131),
 ('нацбаты', 0.6468362808227539),
 ('укронацистов', 0.6359423995018005),
 ('нацики', 0.624232292175293),
 ('всуки', 0.6043425798416138)]

In [284]:
model.save("/Users/katerynaburovova/PycharmProjects/dehumanization/w2v_models/w2v_small_sample_10ep_10w.model")

In [285]:
%%time
w2v_small_sample_20ep_20w = gensim.models.Word2Vec (documents, vector_size=150, window=20, min_count=2, workers=10)
w2v_small_sample_20ep_20w.train(documents,total_examples=len(documents),epochs=20)

CPU times: user 43min 46s, sys: 3min 24s, total: 47min 11s
Wall time: 13min 40s


(666454099, 763560820)

In [306]:
w2v_small_sample_20ep_20w.save("/Users/katerynaburovova/PycharmProjects/dehumanization/w2v_models/w2v_small_sample_20ep_20w.model")

In [296]:
w1 = "украинец" #ы
w2v_small_sample_20ep_20w.wv.most_similar (positive=w1)

[('русских', 0.6968995332717896),
 ('русские', 0.6964977383613586),
 ('украина', 0.651254415512085),
 ('поляк', 0.6197736859321594),
 ('киев', 0.6087530851364136),
 ('европеец', 0.5969522595405579),
 ('русским', 0.5910526514053345),
 ('русскими', 0.585874080657959),
 ('бандеровец', 0.5767456293106079),
 ('нацист', 0.5747347474098206)]

In [303]:
w2 = "укр" #ы
w2v_small_sample_20ep_20w.wv.most_similar (positive=w2)

[('хлопцы', 0.525049090385437),
 ('укрорейха', 0.49813905358314514),
 ('хохол', 0.4795973598957062),
 ('украинский', 0.4786289632320404),
 ('укро', 0.47841185331344604),
 ('укроп', 0.475413054227829),
 ('уничтоженный', 0.4753034710884094),
 ('украинским', 0.47485196590423584),
 ('артой', 0.46757906675338745),
 ('всушник', 0.46454688906669617)]

In [298]:
w3 = "укр" #ы
w2v_small_sample_20ep_20w.wv.most_similar (positive=w3)

[('хлопцы', 0.525049090385437),
 ('укрорейха', 0.49813905358314514),
 ('хохол', 0.4795973598957062),
 ('украинский', 0.4786289632320404),
 ('укро', 0.47841185331344604),
 ('укроп', 0.475413054227829),
 ('уничтоженный', 0.4753034710884094),
 ('украинским', 0.47485196590423584),
 ('артой', 0.46757906675338745),
 ('всушник', 0.46454688906669617)]

In [301]:
w3 = "хлопцы" #ы
w2v_small_sample_20ep_20w.wv.most_similar (positive=w3)

[('хлопец', 0.6090108156204224),
 ('драпать', 0.6059606671333313),
 ('хохлы', 0.5939660668373108),
 ('нацики', 0.5922543406486511),
 ('всушников', 0.5839340686798096),
 ('всушники', 0.578698456287384),
 ('хохол', 0.5765518546104431),
 ('всушник', 0.5621557831764221),
 ('нахй', 0.5591046810150146),
 ('пацаны', 0.5548537969589233)]

In [302]:
w3 = "хохлы" #ы
w2v_small_sample_20ep_20w.wv.most_similar (positive=w3)

[('хохол', 0.6874178051948547),
 ('москали', 0.6082358360290527),
 ('хлопцы', 0.5939660668373108),
 ('блть', 0.592091977596283),
 ('укроп', 0.5853246450424194),
 ('пацаны', 0.5584353804588318),
 ('всушников', 0.541182279586792),
 ('шо', 0.5330790281295776),
 ('свидомые', 0.5313231348991394),
 ('москаль', 0.5298460125923157)]

In [299]:
w4 = "бандеровец" #ы
w2v_small_sample_20ep_20w.wv.most_similar (positive=w4)

[('нацист', 0.7867278456687927),
 ('нациков', 0.6760679483413696),
 ('фашист', 0.6734396815299988),
 ('нацики', 0.6429235935211182),
 ('неонацист', 0.63155597448349),
 ('оккупант', 0.6156263947486877),
 ('бандеры', 0.6109479069709778),
 ('недобиток', 0.5980472564697266),
 ('бандеровскую', 0.5909456610679626),
 ('украинец', 0.5767455697059631)]

In [300]:
w5 = "бандеры" #ы
w2v_small_sample_20ep_20w.wv.most_similar (positive=w5)

[('шухевича', 0.7994802594184875),
 ('бандеру', 0.6805114150047302),
 ('упа', 0.675289511680603),
 ('нацист', 0.6455380916595459),
 ('нацистский', 0.6343949437141418),
 ('бандера', 0.6273082494735718),
 ('бандеровец', 0.6109479069709778),
 ('фашист', 0.5912745594978333),
 ('коллаборационист', 0.5839884877204895),
 ('нацисткой', 0.5792540311813354)]

In [307]:
%%time
w2v_small_sample_30ep_20w = gensim.models.Word2Vec (documents, vector_size=150, window=20, min_count=2, workers=10)
w2v_small_sample_30ep_20w.train(documents,total_examples=len(documents),epochs=30)

CPU times: user 1h 33s, sys: 4min 53s, total: 1h 5min 26s
Wall time: 22min 44s


(999663280, 1145341230)

In [309]:
w2v_small_sample_30ep_20w.save("/Users/katerynaburovova/PycharmProjects/dehumanization/w2v_models/w2v_small_sample_30ep_20w.model")

### Compiling the list of group labels for pre-selection

In [304]:
w = "укры" #ы
w2v_small_sample_20ep_20w.wv.most_similar (positive=w)

[('укров', 0.6507847309112549),
 ('укронацисты', 0.6384262442588806),
 ('хохол', 0.6045107245445251),
 ('всу', 0.5642962455749512),
 ('укроп', 0.5553898215293884),
 ('всушники', 0.5513160824775696),
 ('арта', 0.5499454140663147),
 ('укропов', 0.5253047347068787),
 ('укропские', 0.5176306962966919),
 ('грады', 0.5170502066612244)]

In [313]:
w = "бандеровцы" #ы
model.wv.most_similar (positive=w)

[('ужаснись', 0.7729424834251404),
 ('бердник', 0.6776976585388184),
 ('плине', 0.6214604377746582),
 ('сторя', 0.613357424736023),
 ('пономарь', 0.6017505526542664),
 ('помолитися', 0.6013689041137695),
 ('цибуля', 0.5816124081611633),
 ('окривавлен', 0.5757151246070862),
 ('kurlife', 0.5653677582740784),
 ('послди', 0.56316077709198)]

In [328]:
w = "укрорейх" #ы
w2v_small_sample_30ep_20w.wv.most_similar (positive=w, topn=20)

[('тылы', 0.4570629298686981),
 ('хохлоботов', 0.43998971581459045),
 ('бк', 0.43594059348106384),
 ('укров', 0.43156698346138),
 ('подвисает', 0.427371621131897),
 ('хохол', 0.41582244634628296),
 ('хаймарса', 0.4101269841194153),
 ('химарсов', 0.40948784351348877),
 ('укрорейха', 0.40258699655532837),
 ('укронацистов', 0.40145498514175415),
 ('угил', 0.39898714423179626),
 ('всуки', 0.39834579825401306),
 ('всу', 0.3977126479148865),
 ('укронацисты', 0.3972514867782593),
 ('методично', 0.39680030941963196),
 ('щепки', 0.39532938599586487),
 ('чугуев', 0.393096923828125),
 ('тяо', 0.39104190468788147),
 ('ракетнобомбовые', 0.390695720911026),
 ('нациков', 0.3867169916629791)]

In [329]:
list_of_labels = ['бандеры',
                  'бандеровец',
                  'неонацист',
                  'неонацисты',
                  'хохол',
                  'хохлы',
                  'хлопцы',
                  'укроп',
                  'всушники',
                  'всушник',
                  'свидомые',
                  'укрорейх',
                  'укроп',
                  'укропы',
                  'всуки',
                  'укронацисты',
                  'укронацист',
                  'укронацики',
                  'укронацик',
                  'укро-', #складені імена через дефіс, наскільки я розумію
                  'укровояки',
                  'майданутые',
                  'салорейх',
                  'хрюн',
                  'чуб',
                  ]

In [330]:
list_of_roots = ['бандер',
                  'неонац',
                  'хохол',
                  'хохл',
                  'хлопц',
                  'укроп',
                  'всушник',
                  'свидом',
                  'орейх',
                  'всук',
                  'укронац',
                  'укро-', #складені імена через дефіс, наскільки я розумію
                    'укро ',
                  'укровояк',
                  'майданут',
                  'хрюн',
                  'чуб',
                  ]

In [336]:
%%time
df_sample['mentions_roots'] = df_sample['tokenized_sents'].apply(lambda x: find_sentences_w_sec(x, list_of_roots))

CPU times: user 41.2 s, sys: 4.06 s, total: 45.3 s
Wall time: 54.6 s


In [337]:
df_sample['mentions_roots']

0          []
1          []
2          []
3          []
4          []
           ..
1367304    []
1367305    []
1367306    []
1367307    []
1367308    []
Name: mentions_roots, Length: 1367309, dtype: object

In [338]:
data = df_sample['mentions_roots'].explode().unique()
df_w2v_roots = pd.DataFrame({'text': data[:]})
df_w2v_roots.to_csv('df_w2v_roots.csv')

In [342]:
df_annotation = pd.read_csv('/Users/katerynaburovova/PycharmProjects/dehumanization/ann_dataset.csv', header=0)

In [343]:
df_annotation

,text
0,Вагнер отправляет к бандере и прочим шайтанам в районе Бахмута следующие бригады: Шайтанский батальон Джохара Дудаева. 30-я механизированная бригада. 58-я механизированная пехотная бригада. 53-я механизированная бригада. 4-я бригада быстрого реагирования. 241-я бригада территориальной обороны. 71-я егерская бригада.
1,Тут понимаешь всё про метафизику этой войны. «Укропы» под покровом Запада вышли не против России — они вышли на войну с Богом.
2,"Наш источник сообщает, что остатки военной авиации укрорейха переброшена на аэродром Долгинцево(Кривой Рог)"
3,Раньше укронацисты строили песчаные замки — но теперь пляжи заминированы.
4,Именно эти регионы являются настоящим рассадником укронацизма и имеют для укронациков сакральный идеологический смысл.
...,...
448,"Москва свой полноценный светский сезон вроде отгуляла - остались, простите, кастрированные мероприятия, но Петру Аксёнову в Петербурге собянинский указ не писан."
449,"Джо Байден: мы ограничим возможности России вести бизнес в долларах, фунтах и йенах"
450,"Введение в России массовой сдачи стеклотары поможет снизить ущерб окружающей среде и сократить площади свалок, уверены в заксобрании Ленобласти."
451,В регионе выстраивается система международных связей без участия России.
